In [62]:
import numpy as np
import pandas as pd
import sklearn as sk
import torch
import timeit
from tqdm import tqdm
from sklearn.utils import shuffle

In [63]:
train_data = pd.read_csv("dna-methylation-prediction/train.csv")
train_data0 = train_data[train_data["label"]==0]
train_data1 = train_data[train_data["label"]==1][:5000]
train= train_data0.append(train_data1, ignore_index=False)

train=shuffle(train)
X = train["DNAsequence"]
print(X.head)
y= (train["label"])
print(y.head(20))



<bound method NDFrame.head of 7260     CGTCCTGTTGCTGGTCTCCGTCCGGTCGCCGGCCGTCTAGGTCTCC...
8        GACGTTGGGCAACAAAAAGAAAGTGAAAAGAACCTTACAAAAAATA...
6129     CAGGTGCCCACCACCACACCTGGCTAATTTTTGTAGAGACGGGGTT...
2636     TCTCCTGACCTTGTGATCCACCCGCCTCGGCCTCCCAAAGTGCTGG...
4338     GTGTGCCACCATGCCCGCCTAATTCTTAATATTTTTAGTAGAGATG...
                               ...                        
5218     GGAGTGCAGTGGCGCAATCCCAGCCGACTGCAACCTCTGTCTCCCA...
775      ATAGCTCCTTCCCTCACATTCTTCAGATCTTGGCTCAAATGTCATT...
2299     ACCCTCTATACCAGGGATTCTCAAACTTACTATCATCAAGTCTTTC...
1791     GGGGTTTCACCATGTTGGCCAGGCTAGTCTCAAACTCCTGACATCT...
10184    CAGCAGCGGCTGCGTGCCACCCCACAGAGCGGCCACCAGCACCAGA...
Name: DNAsequence, Length: 10000, dtype: object>
7260     0
8        1
6129     1
2636     1
4338     1
7018     0
3416     1
14486    0
5483     1
21618    0
909      1
2028     1
18639    0
22978    0
7835     0
2776     1
23146    0
4204     1
1572     1
20023    0
Name: label, dtype: int64


In [64]:
y = [float(i) for i in y]
y = torch.Tensor(y)
print(y, y.shape)

tensor([0., 1., 1.,  ..., 1., 1., 0.]) torch.Size([10000])


In [65]:
def transform_DNA(seq):
    mapping = dict(zip("ATCG", range(2, 6)))
    seq2= [mapping[i] for i in seq]
    seq2= [float(i) for i in seq2]
    return np.array(seq2)

In [66]:
print(type(transform_DNA(X[0])[0]))

<class 'numpy.float64'>


In [67]:
Xtrans = transform_DNA(X[0])

for x in tqdm(X[1:]):
    o = transform_DNA(x)
    Xtrans= np.append(Xtrans, o)
    

100%|██████████| 9999/9999 [00:04<00:00, 2222.06it/s]


In [68]:
print(Xtrans.shape)

(2010000,)


In [69]:
X = torch.Tensor([i for i in Xtrans]).view(-1,201)

In [70]:
X.shape

torch.Size([10000, 201])

In [98]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__() 
        self.conv1 = nn.Conv1d(1, 256, 16) 
        self.conv2 = nn.Conv1d(256, 512, 8) 
        self.conv3 = nn.Conv1d(512, 512, 4)

        x = torch.randn(201).view(-1,1,201)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) 
        self.fc2 = nn.Linear(512, 1) 

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool1d(F.relu(self.conv1(x)), (2))
        x = F.max_pool1d(F.relu(self.conv2(x)), (2))
        x = F.max_pool1d(F.relu(self.conv3(x)), (2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, 1,self._to_linear) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        return x


net = Net()
print(net)

Net(
  (conv1): Conv1d(1, 256, kernel_size=(16,), stride=(1,))
  (conv2): Conv1d(256, 512, kernel_size=(8,), stride=(1,))
  (conv3): Conv1d(512, 512, kernel_size=(4,), stride=(1,))
  (fc1): Linear(in_features=10240, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
)


In [99]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
loss_function = nn.MSELoss()

In [100]:
BATCH_SIZE = 64
EPOCHS = 5
a=0
for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = X[i:i+BATCH_SIZE].view(-1, 1, 201)
        batch_y = y[i:i+BATCH_SIZE]
        #print (batch_y)
      

        net.zero_grad()

        outputs = net(batch_X)
        
        #print(outputs.shape, batch_y.shape)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update
        
    print(f"Epoch: {epoch}. Loss: {loss}")

  0%|          | 0/157 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.2512843608856201


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch: 1. Loss: 0.2520515024662018


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch: 2. Loss: 0.25207406282424927


 45%|████▌     | 71/157 [02:04<02:30,  1.75s/it]


KeyboardInterrupt: 